In [2]:
!pip install transformers==4.10.2
!pip install sentencepiece==0.1.96

     |████████████████████████████████| 2.8 MB 9.2 MB/s 
     |████████████████████████████████| 636 kB 45.6 MB/s 
     |████████████████████████████████| 3.3 MB 46.1 MB/s 
     |████████████████████████████████| 895 kB 75.1 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 8.2 MB/s 


In [4]:
!pip install gingerit

     |████████████████████████████████| 62 kB 759 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [12]:
!pip install -U nltk==3.4

     |████████████████████████████████| 1.4 MB 8.0 MB/s 
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436397 sha256=9dde94eae6a6fd3e439f2f420e9bf433c2cbcb20cbdd144a55283584d4ff290f
  Stored in directory: /root/.cache/pip/wheels/13/b8/81/2349be11dd144dc7b68ab983b58cd2fae353cdc50bbdeb09d0
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [35]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
file = open("sample.txt")
text=file.read()
print(text)


People who thught it am just another rainy day, immediated foliated into cats.


In [44]:
from gingerit.gingerit import GingerIt

# text = 'The smelt of fliwers bring back memories.'
ref=text
parser = GingerIt()
first_result=parser.parse(text)['result']

In [45]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5-large-paraphraser-diverse-high-quality")
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)
# Diverse Beam search
context = first_result
text = "paraphrase: "+context + " </s>"
encoding = tokenizer.encode_plus(text,max_length =128, padding=True, return_tensors="pt")
input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
model.eval()
diverse_beam_outputs = model.generate(
    input_ids=input_ids,attention_mask=attention_mask,
    max_length=128,
    early_stopping=True,
    num_beams=5,
    num_beam_groups = 5,
    num_return_sequences=5,
    diversity_penalty = 0.70
)
for beam_output in diverse_beam_outputs:
    second_result = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)


device  cpu


/usr/local/lib/python3.7/dist-packages/transformers/generation_beam_search.py:197: UserWarning:

Passing `max_length` to BeamSearchScorer is deprecated and has no effect.`max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`,or `group_beam_search(...)`.



In [53]:
parser = GingerIt()
third_result=parser.parse(second_result)['result']
third_result = third_result.replace('Paraphrasedoutput: ','')

print("Original Text:")
print(ref)

print("Reduced Text:")
print(third_result)



#-----------------------------------------------------
import os
file2 = open(r"reduced.txt","w")
file2.write(third_result)

Original Text:
People who thught it am just another rainy day, immediated foliated into cats.
Reduced Text:
People who said it was just another rainy day were quickly turned into cats.


76

In [54]:
import re
import nltk
from nltk.util import ngrams, pad_sequence, everygrams
from nltk.tokenize import word_tokenize
from nltk.lm import MLE, WittenBellInterpolated
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter




train_data_file = "sample.txt"


with open(train_data_file) as f:
    train_text = f.read().lower()

# preprocess
train_text = re.sub(r"\[.*\]|\{.*\}", "", train_text)
train_text = re.sub(r'[^\w\s]', "", train_text)

# ngram num
n = 4

# tokenization
training_data = list(pad_sequence(word_tokenize(train_text), n, 
                                  pad_left=True, 
                                  left_pad_symbol="<s>"))

# generate ngrams
ngrams = list(everygrams(training_data, max_len=n))
print("Number of ngrams:", len(ngrams))

# build ngram language models
model = WittenBellInterpolated(n)
model.fit([ngrams], vocabulary_text=training_data)
print(model.vocab)


test_data_file = "reduced.txt"

# Read testing data
with open(test_data_file) as f:
    test_text = f.read().lower()
test_text = re.sub(r'[^\w\s]', "", test_text)

# Tokenize
testing_data = list(pad_sequence(word_tokenize(test_text), n, 
                                 pad_left=True,
                                 left_pad_symbol="<s>"))
print("Length of test data:", len(testing_data))

# scoring
scores = []
for i, item in enumerate(testing_data[n-1:]):
    s = model.score(item, testing_data[i:i+n-1])
    scores.append(s)

scores_np = np.array(scores)

width = 8
height = np.ceil(len(testing_data)/width).astype("int32")
print("Width, Height:", width, ",", height)

# copy scores to rectangular blank array
a = np.zeros(width*height)
a[:len(scores_np)] = scores_np
diff = len(a) - len(scores_np)

a = gaussian_filter(a, sigma=1.0)

a = a.reshape(-1, width)

# format labels
labels = [" ".join(testing_data[i:i+width]) for i in range(n-1, len(testing_data), width)]
labels_individual = [x.split() for x in labels]
labels_individual[-1] += [""]*diff
labels = [f"{x:60.60}" for x in labels]

# create heatmap
fig = go.Figure(data=go.Heatmap(
                z=a, x0=0, dx=1,
                y=labels, zmin=0, zmax=1,
                customdata=labels_individual,
                hovertemplate='%{customdata} <br><b>Score:%{z:.3f}<extra></extra>',
                colorscale="burg"))

fig['layout']['yaxis']['autorange'] = "reversed"
fig.show()

Number of ngrams: 58
<Vocabulary with cutoff=1 unk_label='<UNK>' and 15 items>
Length of test data: 17
Width, Height: 8 , 3


In [56]:
print(a)

[[9.12769015e-01 6.88130645e-01 2.98270619e-01 5.83706017e-02
  4.69323207e-03 4.56441833e-03 5.39760629e-02 2.41903931e-01]
 [3.98832157e-01 2.41903931e-01 5.39760629e-02 4.43062505e-03
  1.33793283e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]]
